In [ ]:
!pip install -q peft bitsandbytes transformers trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 35.0 MB/s eta 0:00:00


In [ ]:
!pip install pyngrok

In [ ]:
!pip install sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 12.9 MB/s eta 0:00:00


In [ ]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 45.8 MB/s eta 0:00:00


In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification,AutoModelForCausalLM
from peft import LoraConfig, PeftModel
import faiss
from sentence_transformers import SentenceTransformer
import re
from datasets import load_dataset
import numpy as np
import streamlit as st

In [ ]:
ds = load_dataset("cardiffnlp/tweet_eval", "emotion")
df = ds["train"].to_pandas()

df

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/233k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/105k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/28.6k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3257 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1421 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/374 [00:00<?, ? examples/s]

,text,label
0,“Worry is a down payment on a problem you may ...,2
1,My roommate: it's okay that we can't spell bec...,0
2,No but that's so cute. Atsu was probably shy a...,1
3,Rooneys fucking untouchable isn't he? Been fuc...,0
4,it's pretty depressing when u hit pan on ur fa...,3
...,...,...
3252,I get discouraged because I try for 5 fucking ...,3
3253,The @user are in contention and hosting @user ...,3
3254,@user @user @user @user @user as a fellow UP g...,0
3255,You have a #problem? Yes! Can you do #somethin...,0


In [ ]:
df=df[:1000]
df.head()

,text,label
0,“Worry is a down payment on a problem you may ...,2
1,My roommate: it's okay that we can't spell bec...,0
2,No but that's so cute. Atsu was probably shy a...,1
3,Rooneys fucking untouchable isn't he? Been fuc...,0
4,it's pretty depressing when u hit pan on ur fa...,3


In [ ]:
df["text"] = df["text"].str.lower()
df.head()

/tmp/ipython-input-1324128920.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"] = df["text"].str.lower()


,text,label
0,“worry is a down payment on a problem you may ...,2
1,my roommate: it's okay that we can't spell bec...,0
2,no but that's so cute. atsu was probably shy a...,1
3,rooneys fucking untouchable isn't he? been fuc...,0
4,it's pretty depressing when u hit pan on ur fa...,3


In [ ]:
def cleaned_text(text):
    text = str(text)
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    text = text.lower()
    text = re.sub(r"\s+", " ", text).strip()
    return text
df["text"]=df["text"].apply(cleaned_text)

/tmp/ipython-input-3660339285.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"]=df["text"].apply(cleaned_text)


In [ ]:
df

,text,label
0,worry is a down payment on a problem you may n...,2
1,my roommate its okay that we cant spell becaus...,0
2,no but thats so cute atsu was probably shy abo...,1
3,rooneys fucking untouchable isnt he been fucki...,0
4,its pretty depressing when u hit pan on ur fav...,3
...,...,...
995,user directtv is the best apparently you just ...,0
996,ok but how are the fast amp furious movies not...,1
997,user terrible paul so deserved that win bbfail,1
998,the are playing antichrist why wont user play ...,0


In [ ]:
df["label"]

,label
0,2
1,0
2,1
3,0
4,3
...,...
995,0
996,1
997,1
998,0


In [ ]:
label_map = {
    0: "anger",
    1: "joy",
    2: "optimism",
    3: "sadness"
}
df["label"] = df["label"].map(label_map)

/tmp/ipython-input-3455426269.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["label"] = df["label"].map(label_map)


In [ ]:
df.head()

,text,label
0,worry is a down payment on a problem you may n...,optimism
1,my roommate its okay that we cant spell becaus...,anger
2,no but thats so cute atsu was probably shy abo...,joy
3,rooneys fucking untouchable isnt he been fucki...,anger
4,its pretty depressing when u hit pan on ur fav...,sadness


In [ ]:
df.to_json("dj", orient="values", force_ascii=False)

In [ ]:
dj = df.to_dict(orient="records")
texts = [row["text"] for row in dj]
texts[:2]

['worry is a down payment on a problem you may never have joyce meyer motivation leadership worry',
 'my roommate its okay that we cant spell because we have autocorrect terrible firstworldprobs']

In [ ]:
clf_tokenizer = AutoTokenizer.from_pretrained("bhadresh-savani/distilbert-base-uncased-emotion")
clf_model = AutoModelForSequenceClassification.from_pretrained("bhadresh-savani/distilbert-base-uncased-emotion")
gen_model = AutoModelForCausalLM.from_pretrained("gpt2")
gen_tokenizer=AutoTokenizer.from_pretrained("gpt2")

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
model_rag = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
embeddings = model_rag.encode(texts, show_progress_bar=True)
embeddings = np.array(embeddings)

print("Embeddings shape:", embeddings.shape)

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Embeddings shape: (1000, 384)


In [ ]:
index = faiss.IndexFlatL2(embeddings.shape[1])

index.add(embeddings)
def retrieve_documents(query, k=2):
    # Encode the query
    query_embedding = model_rag.encode([query])
    # Search the index for the top-k most similar documents
    distances, indices = index.search(query_embedding, k)
    # Return the relevant documents
    return [texts[i] for i in indices[0]]

query = "i am happy "
retrieved_docs = retrieve_documents(query)
print("Retrieved documents:")
for doc in retrieved_docs:
    print(f"- {doc}")

Retrieved documents:
- or when someone tells me i needa smile like excuse me now im just frowning even harder are you happy
- user cheer up


In [ ]:
def classify_emotion(text):
    inputs = clf_tokenizer(text, return_tensors="pt")
    outputs = clf_model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    pred = torch.argmax(probs, dim=1).item()
    return label_map[pred], probs[0][pred].item()
classify_emotion("i am happy")

('joy', 0.998437225818634)

In [ ]:
def rag_with_bert_and_gpt2(query, k=2):
    emotion, confidence = classify_emotion(query)

    retrieved_docs = retrieve_documents(query, k)
    context = " ".join(retrieved_docs)

    prompt = f"""
    The user is feeling {emotion}.
    Use only the following documents to answer.
    Documents: {context}

    Question: {query}
    Answer:
    """
    inputs = gen_tokenizer.encode(prompt, return_tensors="pt")
    outputs = gen_model.generate(
        inputs,
        max_length=150,
        num_return_sequences=1,
        early_stopping=True
    )
    answer = gen_tokenizer.decode(outputs[0], skip_special_tokens=True)

    return {
        "retrieved_docs": retrieved_docs,
        "answer": answer,
        "emotion": emotion,
        "confidence": confidence
    }


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'retrieved_docs': ['or when someone tells me i needa smile like excuse me now im just frowning even harder are you happy',
  'user cheer up'],
 'answer': '\n    The user is feeling joy.\n    Use only the following documents to answer.\n    Documents: or when someone tells me i needa smile like excuse me now im just frowning even harder are you happy user cheer up\n\n    Question: i am happy\n    Answer:\n                                                                                 ',
 'emotion': 'joy',
 'confidence': 0.998437225818634}

In [ ]:
def generate_fewshot(user_input):
    examples = (
    "Task: Given a text, predict its emotion.\n\n"

    "Text: I am very sad today.\n"
    "Label: sadness\n\n"

    "Text: Wow, I just won the game!\n"
    "Label: joy\n\n"

    "Text: This makes me so angry.\n"
    "Label: anger\n\n"

    "Text: I feel so relaxed and loved.\n"
    "Label: love\n\n"

    "Text: That was shocking!\n"
    "Label: surprise\n\n"

    "Text: I am worried about tomorrow.\n"
    "Label: fear\n\n"
    )
    prompt = examples + f"User: {user_input}\nBot:"

    inputs = gen_tokenizer(prompt, return_tensors="pt")
    outputs = gen_model.generate(
        **inputs,
        max_length=128,
        do_sample=True,
        top_k=50,
        num_return_sequences=1,
        repetition_penalty=1.2
    )
    response = gen_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.strip()

In [ ]:
%%writefile chatbot_app.py
import faiss
from datasets import load_dataset
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
import re
from transformers import AutoTokenizer, AutoModelForSequenceClassification,AutoModelForCausalLM
import streamlit as st
ds = load_dataset("cardiffnlp/tweet_eval", "emotion")
df = ds["train"].to_pandas()
dj = df.to_dict(orient="records")

label_map = {
    0: "anger",
    1: "joy",
    2: "optimism",
    3: "sadness"
}
texts = [row["text"] for row in dj]
texts[:2]
model_rag = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embeddings = model_rag.encode(texts, show_progress_bar=True)
embeddings = np.array(embeddings)
clf_tokenizer = AutoTokenizer.from_pretrained("bhadresh-savani/distilbert-base-uncased-emotion")
clf_model = AutoModelForSequenceClassification.from_pretrained("bhadresh-savani/distilbert-base-uncased-emotion")
gen_model = AutoModelForCausalLM.from_pretrained("gpt2")
gen_tokenizer=AutoTokenizer.from_pretrained("gpt2")
model_rag = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
def classify_emotion(text):
    inputs = clf_tokenizer(text, return_tensors="pt")
    outputs = clf_model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    pred = torch.argmax(probs, dim=1).item()
    return label_map[pred], probs[0][pred].item()
classify_emotion("i am happy")
index = faiss.IndexFlatL2(embeddings.shape[1])

index.add(embeddings)
def retrieve_documents(query, k=2):
    # Encode the query
    query_embedding = model_rag.encode([query])
    # Search the index for the top-k most similar documents
    distances, indices = index.search(query_embedding, k)
    # Return the relevant documents
    return [texts[i] for i in indices[0]]

def rag_with_bert_and_gpt2(query, k=2):
    emotion, confidence = classify_emotion(query)

    retrieved_docs = retrieve_documents(query, k)
    context = " ".join(retrieved_docs)

    prompt = f"""
    The user is feeling {emotion}.
    Use only the following documents to answer.
    Documents: {context}

    Question: {query}
    Answer:
    """
    inputs = gen_tokenizer.encode(prompt, return_tensors="pt")
    outputs = gen_model.generate(
        inputs,
        max_length=150,
        num_return_sequences=1,
        early_stopping=True
    )
    answer = gen_tokenizer.decode(outputs[0], skip_special_tokens=True)

    return {
        "retrieved_docs": retrieved_docs,
        "answer": answer,
        "emotion": emotion,
        "confidence": confidence
    }
st.title("")
st.write("This chatbot explains concepts in simple words using few-shot learning.")

user_input = st.text_input("You:")
if st.button("Send") and user_input:
    st.write("Bot:", rag_with_bert_and_gpt2(user_input))

Overwriting chatbot_app.py


In [ ]:
!ngrok config add-authtoken 32WErQmZ3j7ITLZAI86vHGX8dQA_cSyGuXUxnBd6HmT2GpJo

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!kill -9 $(pgrep ngrok) >/dev/null 2>&1 || true

In [ ]:
from pyngrok import ngrok
!streamlit run chatbot_app.py &>/dev/null &
url = ngrok.connect(8501)
print('Chatbot running at:', url)

Chatbot running at: NgrokTunnel: "https://3e4f0efb54b8.ngrok-free.app" -> "http://localhost:8501"
